# Utilities
> Build utilities for flask

In [1]:
# default_exp utils

In [14]:
# export
import os
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [3]:
# export
HOME = Path(os.environ["HOME"])
EMBER = HOME/".torchember"
DATA = EMBER/"data"
LOG = EMBER/"LOG"
def check_existance():
    if DATA.exists()==False:
        return False
    if LOG.exists()==False:
        return False
    else:
        return True
    

Check if the data file exists, if not, use torch ember in your pytorch modeling first according to the short tutorial

In [4]:
check_existance()

True

In [5]:
# export
def get_ember_list():
    if check_existance()== False:
        return None
    else:
        return  list(i for i in os.listdir(DATA) if i[:5]=="base_")

In [6]:
get_ember_list()[:5]

['base_AlexNet_20200215_132629.json',
 'base_AlexNet_20200211_230327.json',
 'base_AlexNet_20200211_192928.json',
 'base_AlexNet_20200209_183014.json',
 'base_AlexNet_20200211_193437.json']

In [7]:
# export
def unpack_meta(fname):
    f = open(DATA/fname,"r")
    dict_ = json.loads(f.read())
    f.close()
    dict_["name"] = fname[5:-5]
    return dict_

def get_ember_df(ember_list):
    df = pd.DataFrame(list(unpack_meta(i) for i in ember_list))
    df = df.sort_values(by = "start",ascending = False)
    return df.reset_index().drop("index",axis=1)

In [8]:
df = get_ember_df(get_ember_list())
df.head(5)

,start,user,name
0,2020-02-25 08:21:27,salvor,AlexNet_20200225_082127
1,2020-02-24 23:32:07,salvor,AlexNet_20200224_233207
2,2020-02-24 23:28:27,salvor,AlexNet_20200224_232827
3,2020-02-24 22:46:54,salvor,AlexNet_20200224_224654
4,2020-02-24 21:40:30,salvor,AlexNet_20200224_214030


### Ember Reader
A reading handler to process Torch Ember data

In [9]:
# export
from torchember.core import emberTracker

class emberReader(object):
    def __init__(self, name):
        self.name = name
        self.t = emberTracker(name)
        self.structure = self.t[f"structure_{self.name}"]
        self.base = self.t[f"base_{self.name}"]
        
    @property
    def latest(self):
        return self.t.latest_df.to_dict(orient = "record")

In [19]:
er = emberReader("AlexNet_20200225_082127")

In [20]:
er.structure

{'name': 'model(AlexNet)',
 'short': 'model(AlexNet)',
 'children': [{'name': 'model(AlexNet).features(Sequential)',
   'short': 'features(Sequential)',
   'children': [{'name': 'model(AlexNet).features(Sequential).0(Conv2d)',
     'short': '0(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).1(ReLU)',
     'short': '1(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).2(MaxPool2d)',
     'short': '2(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).3(Conv2d)',
     'short': '3(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).4(ReLU)',
     'short': '4(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).5(MaxPool2d)',
     'short': '5(MaxPool2d)'},
    {'name': 'model(AlexNet).features(Sequential).6(Conv2d)',
     'short': '6(Conv2d)'},
    {'name': 'model(AlexNet).features(Sequential).7(ReLU)',
     'short': '7(ReLU)'},
    {'name': 'model(AlexNet).features(Sequential).8(Conv2d)',
     'short': '8(Conv2d)'},
    {'name': 'model(AlexN

In [21]:
er.base

{'start': '2020-02-25 08:21:27', 'user': 'salvor'}

In [22]:
er.t.latest_df

,num,row_min,module,ts,ttype,tname,row_max
0,301056,"[-1.9999966621398926, -1.9999827146530151]",model(AlexNet),2020-02-25 08:22:06,input,x,NaN
1,301056,"[-1.9999966621398926, -1.9999827146530151]",model(AlexNet).features(Sequential),2020-02-25 08:22:06,input,input,NaN
2,301056,"[-1.9999966621398926, -1.9999827146530151]",model(AlexNet).features(Sequential).0(Conv2d),2020-02-25 08:22:06,input,input,NaN
3,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-25 08:22:06,weight,weight_0,"[0.05191058665513992, 0.05214138329029083, 0.0..."
4,23232,NaN,model(AlexNet).features(Sequential).0(Conv2d),2020-02-25 08:22:06,weight_grad,grad_0,"[4.042061118525453e-05, 5.2498202421702445e-05..."
...,...,...,...,...,...,...,...
75,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-25 08:22:06,weight,weight_1,"[-0.003977091982960701, 0.007577169686555862, ..."
76,1000,NaN,model(AlexNet).classifier(Sequential).6(Linear),2020-02-25 08:22:06,weight_grad,grad_1,"[0.003000000026077032, 0.003000000026077032, 0..."
77,2000,"[-0.03259588032960892, -0.03573821857571602]",model(AlexNet).classifier(Sequential).6(Linear),2020-02-25 08:22:06,output,output_0,NaN
78,2000,"[-0.03259588032960892, -0.03573821857571602]",model(AlexNet).classifier(Sequential),2020-02-25 08:22:06,output,output_0,NaN


In [23]:
er.latest

[{'num': 301056,
  'row_min': [-1.9999966621398926, -1.9999827146530151],
  'module': 'model(AlexNet)',
  'ts': '2020-02-25 08:22:06',
  'ttype': 'input',
  'tname': 'x',
  'row_max': nan},
 {'num': 301056,
  'row_min': [-1.9999966621398926, -1.9999827146530151],
  'module': 'model(AlexNet).features(Sequential)',
  'ts': '2020-02-25 08:22:06',
  'ttype': 'input',
  'tname': 'input',
  'row_max': nan},
 {'num': 301056,
  'row_min': [-1.9999966621398926, -1.9999827146530151],
  'module': 'model(AlexNet).features(Sequential).0(Conv2d)',
  'ts': '2020-02-25 08:22:06',
  'ttype': 'input',
  'tname': 'input',
  'row_max': nan},
 {'num': 23232,
  'row_min': nan,
  'module': 'model(AlexNet).features(Sequential).0(Conv2d)',
  'ts': '2020-02-25 08:22:06',
  'ttype': 'weight',
  'tname': 'weight_0',
  'row_max': [0.05191058665513992,
   0.05214138329029083,
   0.05229843407869339,
   0.0524069145321846,
   0.05245247483253479,
   0.05230458080768585,
   0.05164984613656998,
   0.05228611826896667